In [ ]:
# Check if the easydiffraction library is installed.
# If not, install it with the 'visualization' extras.
# Needed when running remotely (e.g. Colab) where the lib is absent.
import builtins
import importlib.util

if (hasattr(builtins, '__IPYTHON__') and
    importlib.util.find_spec('easydiffraction') is None):
    !pip install 'easydiffraction[visualization]'


# Pair Distribution Function: Si, NPD

This example demonstrates a pair distribution function (PDF) analysis
of Si, based on data collected from a time-of-flight neutron powder
diffraction experiment at NOMAD at SNS.

## Import Library

In [ ]:
import easydiffraction as ed

## Create Project

In [ ]:
project = ed.Project()

## Set Plotting Engine

In [ ]:
# Keep the auto-selected engine. Alternatively, you can uncomment the
# line below to explicitly set the engine to the required one.
# project.plotter.engine = 'plotly'

In [ ]:
# Set global plot range for plots
project.plotter.x_max = 40

## Add Sample Model

In [ ]:
project.sample_models.add(name='si')

In [ ]:
sample_model = project.sample_models['si']
sample_model.space_group.name_h_m.value = 'F d -3 m'
sample_model.space_group.it_coordinate_system_code = '1'
sample_model.cell.length_a = 5.43146
sample_model.atom_sites.add(
    label='Si',
    type_symbol='Si',
    fract_x=0,
    fract_y=0,
    fract_z=0,
    wyckoff_letter='a',
    b_iso=0.5,
)

## Add Experiment

In [ ]:
data_path = ed.download_data(id=5, destination='data')

In [ ]:
project.experiments.add(
    name='nomad',
    data_path=data_path,
    sample_form='powder',
    beam_mode='time-of-flight',
    radiation_probe='neutron',
    scattering_type='total',
)

In [ ]:
experiment = project.experiments['nomad']
experiment.linked_phases.add(id='si', scale=1.0)
experiment.peak.damp_q = 0.02
experiment.peak.broad_q = 0.03
experiment.peak.cutoff_q = 35.0
experiment.peak.sharp_delta_1 = 0.0
experiment.peak.sharp_delta_2 = 4.0
experiment.peak.damp_particle_diameter = 0

## Select Fitting Parameters

In [ ]:
project.sample_models['si'].cell.length_a.free = True
project.sample_models['si'].atom_sites['Si'].b_iso.free = True
experiment.linked_phases['si'].scale.free = True

In [ ]:
experiment.peak.damp_q.free = True
experiment.peak.broad_q.free = True
experiment.peak.sharp_delta_1.free = True
experiment.peak.sharp_delta_2.free = True

## Run Fitting

In [ ]:
project.analysis.current_calculator = 'pdffit'
project.analysis.fit()

## Plot Measured vs Calculated

In [ ]:
project.plot_meas_vs_calc(expt_name='nomad', show_residual=False)